To directly pass a script retrieved from MinIO to a Docker container for execution and then stream the output back as an annotated response, we'll need to adjust the workflow to include:

1. **Retrieving the script** from MinIO.
2. **Executing the script inside a Docker container**.
3. **Capturing the script's output** and streaming it back in the response.

This process involves using the Docker SDK for Python to manage container execution and the MinIO Python client (`minio`) for accessing scripts in MinIO storage. Note that to fully implement this, you'll need appropriate error handling and security measures, which are simplified in this example for brevity.

First, ensure you have the Docker SDK and MinIO client installed:



In [1]:
!pip install docker minio

DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063



### API Implementation with FastAPI

Here's an example FastAPI application that demonstrates this process:

In [3]:
from fastapi import FastAPI, HTTPException
from minio import Minio
from docker import from_env as docker_from_env
from docker.models.containers import Container
import tempfile

app = FastAPI()

# Configure MinIO client
minio_client = Minio(
    "cda-DESKTOP:9000",
    access_key="cda_cdaprod",
    secret_key="cda_cdaprod",
    secure=False
)

# Configure Docker client
docker_client = docker_from_env()

@app.post("/execute")
async def execute_script(script_name: str):
    try:
        # Retrieve script from MinIO
        response = minio_client.get_object("notebooks-data-bucket", script_name)
        script_content = response.read().decode('utf-8')

        # Create a temporary file to store the script
        with tempfile.NamedTemporaryFile(suffix=".py", delete=False) as script_file:
            script_file.write(script_content.encode())
            script_path = script_file.name

        # Execute the script inside a Docker container
        container: Container = docker_client.containers.run(
            image="python:3.8-slim",  # Ensure this image has necessary packages installed
            command=f"python {script_path}",
            volumes={script_path: {'bind': script_path, 'mode': 'ro'}},
            detach=True,
            remove=True
        )

        # Wait for the container to finish and capture output
        result = container.wait()
        output = container.logs().decode('utf-8')

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

    return {"script_output": output}

In [ ]:
!pip install minio

In [1]:
from minio import Minio
from tempfile import NamedTemporaryFile
import os

# Assuming minio_client is already initiated
# Example: minio_client = Minio("play.min.io", access_key="yourAccessKey", secret_key="yourSecretKey", secure=True)
minio_client = Minio(
    "cda-DESKTOP:9000",
    access_key="cda_cdaprod",
    secret_key="cda_cdaprod",
    secure=False
)

# Define the content of your Python script
python_script_content = """
def say_hello(name):
    print(f"Hello, {name}!")

if __name__ == "__main__":
    say_hello("World")
"""

# Create a temporary file to write the script content
with NamedTemporaryFile(delete=False, suffix=".py") as temp_file:
    temp_file.write(python_script_content.encode())
    temp_file_path = temp_file.name  # Save the path for later use

# Upload the file to the MinIO bucket "python"
bucket_name = "python"
object_name = os.path.basename(temp_file_path)  # Use the file name as the object name in MinIO

try:
    # Ensure the bucket exists
    if not minio_client.bucket_exists(bucket_name):
        minio_client.make_bucket(bucket_name)
    
    # Upload the file
    minio_client.fput_object(bucket_name, object_name, temp_file_path)
    print(f"Uploaded '{object_name}' to bucket '{bucket_name}'.")
finally:
    # Clean up the temporary file
    os.remove(temp_file_path)

Uploaded 'tmpgedq6yo8.py' to bucket 'python'.


In [ ]:
# Import necessary libraries
from fastapi import FastAPI, HTTPException
from minio import Minio
from docker import from_env as docker_from_env
import tempfile
import uvicorn
import nest_asyncio

# Apply nest_asyncio patch
nest_asyncio.apply()

# Initialize FastAPI app
app = FastAPI()

# Configure MinIO client
minio_client = Minio(
    "cda-DESKTOP:9000",
    access_key="cda_cdaprod",
    secret_key="cda_cdaprod",
    secure=False
)

# Configure Docker client
docker_client = docker_from_env()

# Define the FastAPI endpoint
@app.post("/execute/{script_name}")
async def execute_script(script_name: str):
    try:
        # Retrieve script from MinIO
        response = minio_client.get_object("python", script_name)
        script_content = response.read().decode('utf-8')

        # Create a temporary file to store the script
        with tempfile.NamedTemporaryFile(suffix=".py", delete=False) as script_file:
            script_file.write(script_content.encode())
            script_path = script_file.name

        # Execute the script inside a Docker container
        container = docker_client.containers.run(
            image="python:3.8-slim",  # Ensure this image has necessary packages installed
            command=f"python {script_path}",
            volumes={script_path: {'bind': script_path, 'mode': 'ro'}},
            detach=True,
            remove=True
        )

        # Wait for the container to finish and capture output
        result = container.wait()
        output = container.logs().decode('utf-8')

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

    return {"script_output": output}

# Run the FastAPI app within the notebook
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [45112]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     192.168.0.10:52594 - "POST /execute/tmpgedq6yo8.py HTTP/1.1" 500 Internal Server Error


### Key Points:

- **Temporary File**: The script retrieved from MinIO is stored in a temporary file. This file is then mounted and executed inside a Docker container.
- **Docker Execution**: We use the `docker_client.containers.run` method to run the script inside a container based on a Python image. Adjust the image as needed to include any dependencies your scripts may require.
- **Output Capture**: After the container finishes execution, we capture its logs as the script's output.
- **Security and Cleanup**: Ensure the temporary file is securely handled and appropriately cleaned up after use. Additionally, consider security implications of executing arbitrary code and apply necessary safeguards.

### Enhancements for Production Use:

- **Authentication and Permissions**: Add authentication for the API and validate permissions for accessing specific scripts in MinIO.
- **Error Handling**: Implement comprehensive error handling, including handling cases where the script execution fails or produces errors.
- **Performance and Scaling**: Consider performance implications, especially for long-running scripts, and explore options for scaling the execution environment as needed.

This example provides a foundation for building a system that executes Python scripts stored in MinIO on-demand, using Docker for execution isolation and streaming the output back through an API.